In [ ]:
def readFile(fileName):
    lines = []
    with open(fileName, 'r') as f:
        for line in f:
            line = line.strip()
            # Skip empty lines or comment lines
            if (not line or line.startswith('#')):
                continue
            lines.append(line)

    # First non-comment line should be 'P2'
    if (lines[0] != 'P2'):
        raise ValueError("Not a valid P2 file.")

    # Next line has xAxis and yAxis
    XY = lines[1].split()
    xAxis = int(XY[0])
    yAxis = int(XY[1])

    # Next line has maxValue
    maxValue = int(lines[2])

    # Remaining lines have the pixel data
    pixel_tokens = " ".join(lines[3:]).split()
    if (len(pixel_tokens) != xAxis * yAxis):
        raise ValueError("Pixel data size does not match image dimensions.")

    image = []
    idx = 0
    for _ in range(yAxis):
        row = []
        for _ in range(xAxis):
            row.append(int(pixel_tokens[idx]))
            idx += 1
        image.append(row)
    return image, xAxis, yAxis, maxValue

def writeToFile(fileName, image, xAxis, yAxis, maxValue):
    """Writes a 2D list (image) to a P2 .pgm file with given dimensions."""
    with open(fileName, 'w') as f:
        f.write("P2\n")
        f.write(f"{xAxis} {yAxis}\n")
        f.write(f"{maxValue}\n")
        for row in image:
            f.write(" ".join(map(str, row)) + "\n")

def calculateEnergy(image, xAxis, yAxis):
    """Returns a 2D list of energy values computed from the neighboring differences."""
    energy = [[0]*xAxis for _ in range(yAxis)]
    for j in range(yAxis):
        for i in range(xAxis):
            val = image[j][i]
            up    = image[j-1][i] if j > 0 else val
            down  = image[j+1][i] if j < yAxis-1 else val
            left  = image[j][i-1] if i > 0 else val
            right = image[j][i+1] if i < xAxis-1 else val
            energy[j][i] = abs(val - up) + abs(val - down) + abs(val - left) + abs(val - right)
    return energy

def calculateCumulativeVertical(energy, xAxis, yAxis):
    """Builds the cumulative energy for vertical seams from top (row 0) to bottom (row yAxis-1)."""
    M = [[energy[r][c] for c in range(xAxis)] for r in range(yAxis)]
    for j in range(1, yAxis):
        for i in range(xAxis):
            best = M[j-1][i]
            if i > 0:
                best = min(best, M[j-1][i-1])
            if i < xAxis-1:
                best = min(best, M[j-1][i+1])
            M[j][i] += best
    return M

def findVerticalSeam(M, xAxis, yAxis):
    """Traces the lowest-energy vertical seam from bottom to top."""
    seam = [-1]*yAxis
    # Find min in last row
    lastrow = M[yAxis-1]
    min_index = 0
    min_val = lastrow[0]
    for i in range(1, xAxis):
        if (lastrow[i] < min_val):
            min_val = lastrow[i]
            min_index = i
    seam[yAxis-1] = min_index
    # Trace upwards
    for j in range(yAxis-2, -1, -1):
        i = seam[j+1]
        col_candidates = [i]
        if (i > 0):
            col_candidates.append(i-1)
        if (i < xAxis-1):
            col_candidates.append(i+1)
        # pick left-most min from the row above
        best = col_candidates[0]
        for c in col_candidates:
            if (M[j][c] < M[j][best]):
                best = c
        seam[j] = best
    return seam

def removeVerticalSeam(image, seam, xAxis, yAxis):
    """Removes pixels along the vertical seam."""
    for j in range(yAxis):
        i = seam[j]
        image[j].pop(i)
    return image

def remove_n_vertical_seams(image, xAxis, yAxis, n):
    """Removes n vertical seams from the image."""
    for _ in range(n):
        energy = calculateEnergy(image, xAxis, yAxis)
        M = calculateCumulativeVertical(energy, xAxis, yAxis)
        seam = findVerticalSeam(M, xAxis, yAxis)
        removeVerticalSeam(image, seam, xAxis, yAxis)
        xAxis -= 1
    return image, xAxis, yAxis

def transpose(image):
    return list(map(list, zip(*image)))

def remove_n_horizontal_seams(image, xAxis, yAxis, n):
    """Removes n horizontal seams (by transposing, removing vertical, then transposing back)."""
    # Transpose the image
    image_t = transpose(image)
    # Now remove vertical seams from transposed
    image_t, newY, newX = remove_n_vertical_seams(image_t, yAxis, xAxis, n)
    # Transpose back
    image_new = transpose(image_t)
    return image_new, newX, newY

# Example usage in new cells:
# Part I
filename = "bug.pgm"
verticalSeam2Remove = 10
horizontalSeam2Remove = 8

filename1 = filename.split(".")[0]+"_processed_"+str(verticalSeam2Remove)+"_0.pgm"
print(filename1)
data, xAxis, yAxis, maxValue = readFile(filename)
data, xAxis, yAxis = remove_n_vertical_seams(data, xAxis, yAxis, verticalSeam2Remove)
writeToFile(filename1, data, xAxis, yAxis, maxValue)

# Part II
filename2 = filename.split('.')[0] + "_processed_" + str(verticalSeam2Remove) + "_" + str(horizontalSeam2Remove) + ".pgm"
data, xAxis, yAxis, maxValue = readFile(filename)  # reload original
data, xAxis, yAxis = remove_n_vertical_seams(data, xAxis, yAxis, verticalSeam2Remove)
data, xAxis, yAxis = remove_n_horizontal_seams(data, xAxis, yAxis, horizontalSeam2Remove)
writeToFile(filename2, data, xAxis, yAxis, maxValue)

bug_processed_10_0.pgm
